**資料庫**：

1. 自行在本機安裝 SQL Server 2019 Express [https://www.microsoft.com/zh-TW/download/details.aspx?id=101064](https://www.microsoft.com/zh-TW/download/details.aspx?id=101064)
2. 免安裝線上練習資源： [https://sqliteonline.com/](https://sqliteonline.com/)
3. Azure 雲端 SQL Server [https://azure.microsoft.com/zh-tw/free/sql-on-azure/](https://azure.microsoft.com/zh-tw/free/sql-on-azure/)
4. Google Cloud SQL Server [https://cloud.google.com/sql-server](https://cloud.google.com/sql-server)

**問題1**：為什麼要使用 Chinese\_Taiwan\_Stroke\_CI\_AS 定序與 Unicode 支援

<span class="s1" style="color: rgb(0, 0, 0);">參考：</span>[https://docs.microsoft.com/zh-tw/sql/relational-databases/collations/collation-and-unicode-support?view=sql-server-ver15](https://docs.microsoft.com/zh-tw/sql/relational-databases/collations/collation-and-unicode-support?view=sql-server-ver15)

Chinese-Taiwan-Stroke、排序不區分大小寫、區分腔調

<span class="s2" style="font-variant-numeric: normal; font-variant-east-asian: normal; font-stretch: normal; line-height: normal; font-family: &quot;PingFang TC&quot;;">同時在使用</span>  NCHAR<span class="s2" style="font-variant-numeric: normal; font-variant-east-asian: normal; font-stretch: normal; line-height: normal; font-family: &quot;PingFang TC&quot;;">、</span>NVARCHAR <span class="s2" style="font-variant-numeric: normal; font-variant-east-asian: normal; font-stretch: normal; line-height: normal; font-family: &quot;PingFang TC&quot;;">和</span> NTEXT <span class="s2" style="font-variant-numeric: normal; font-variant-east-asian: normal; font-stretch: normal; line-height: normal; font-family: &quot;PingFang TC&quot;;">資料類型時能支援</span>  unicode <span class="s2" style="font-variant-numeric: normal; font-variant-east-asian: normal; font-stretch: normal; line-height: normal; font-family: &quot;PingFang TC&quot;;">及</span> emoji

In [ ]:
-- Warnng:建立資料庫 -- 
CREATE DATABASE [shop]
    COLLATE Chinese_Taiwan_Stroke_CI_AS;  
GO

In [ ]:
-- 切換使用資料庫 --
USE [shop];
GO

**公司類型**

通路商 channel

代理商 agent

零售 retailer

品牌 brand

製造 manufacture

網路商店 shop

In [ ]:
-- 因為公司主鍵 ID 同時繫結了商店，作為其外部索引鍵，所以要刪除公司資料表前，需先刪除此外部索引 -- 
ALTER TABLE [dbo].[Stores]
DROP CONSTRAINT [fk_CompanyID];
GO

-- Warning：刪除[公司]資料表 --
/*
IF  EXISTS (SELECT * FROM sys.objects WHERE object_id = OBJECT_ID(N'[dbo].[Companies]') AND type in (N'U'))
DROP TABLE [dbo].[Companies]
GO
*/
DROP TABLE IF EXISTS [dbo].[Companies]
GO

-- 建立[公司]資料表 --
CREATE TABLE [dbo].[Companies] (
    [ID]                BIGINT         IDENTITY (1, 1) NOT NULL,
    [CompanyName]       NVARCHAR (100) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [CompanySupervisor] BIGINT         DEFAULT (0) NOT NULL,
    [CompanyDesc]       NVARCHAR (MAX) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [CompanyType]       VARCHAR (20)   DEFAULT ('retailer') NOT NULL,
    [CountryCode]       VARCHAR (10)   DEFAULT ('') NOT NULL,
    [TaxID]             VARCHAR (20)   DEFAULT ('') NOT NULL,
    [Url]               VARCHAR (100)  DEFAULT ('') NOT NULL,
    [Email]             VARCHAR (100)  DEFAULT ('') NOT NULL,
    [Phone]             VARCHAR (100)  DEFAULT ('') NOT NULL,
    [Created]         DATETIME       DEFAULT (getdate()) NOT NULL,
    [Modified]   DATETIME       DEFAULT (getdate()) NOT NULL,
    [Status]            TINYINT        DEFAULT ((0)) NOT NULL,
    PRIMARY KEY CLUSTERED ([ID] ASC)
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_CompanyName]
    ON [dbo].[Companies]([CompanyName] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_CountryCode]
    ON [dbo].[Companies]([CountryCode] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_TaxID]
    ON [dbo].[Companies]([TaxID] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_Url]
    ON [dbo].[Companies]([Url] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_Email]
    ON [dbo].[Companies]([Email] ASC);
GO

In [ ]:
-- Warning：刪除[公司詮釋資料]資料表 --
DROP TABLE IF EXISTS [dbo].[CompanyMeta]
GO

-- 建立[公司詮釋資料]資料表 --
CREATE TABLE [dbo].[CompanyMeta] (
    [CMetaID]   BIGINT         IDENTITY (1, 1) NOT NULL,
    [CompanyID] BIGINT         NOT NULL,
    [MetaKey]   VARCHAR (255)  DEFAULT ('') NULL,
    [MetaValue] NVARCHAR (MAX) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NULL,
    PRIMARY KEY CLUSTERED ([CMetaID] ASC)
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_CompanyID]
    ON [dbo].[CompanyMeta]([CompanyID] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_CompanyMetaKey]
    ON [dbo].[CompanyMeta]([MetaKey] ASC);
GO

In [ ]:
-- Warning：刪除[商店]資料表 --
DROP TABLE IF EXISTS [dbo].[Stores]
GO

-- 建立[商店]資料表 --
CREATE TABLE [dbo].[Stores] (
    [ID]              BIGINT         IDENTITY (1, 1) NOT NULL,
    [CompanyID]       BIGINT         DEFAULT (0) NOT NULL,
    [StoreName]       NVARCHAR (100) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [StoreSupervisor] BIGINT         DEFAULT (0) NOT NULL,
    [StoreDesc]       NVARCHAR (MAX) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [Phone]           VARCHAR (100)  DEFAULT ('') NOT NULL,
    [CountryCode]     VARCHAR (10)   DEFAULT ('') NOT NULL,
    [ZipCode]         VARCHAR (10)   DEFAULT ('') NOT NULL,
    [State]           NVARCHAR (30)  DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [City]            NVARCHAR (40)  DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [Address]         NVARCHAR (200) DEFAULT (('') collate Chinese_Taiwan_Stroke_CI_AS) NOT NULL,
    [Created]       DATETIME       DEFAULT (getdate()) NOT NULL,
    [Modified]   DATETIME       DEFAULT (getdate()) NOT NULL,
    [Status]          TINYINT        DEFAULT ((0)) NOT NULL,
    PRIMARY KEY CLUSTERED ([ID] ASC),
    CONSTRAINT [fk_CompanyID] FOREIGN KEY ([CompanyID]) REFERENCES [dbo].[Companies] ([ID])
);
GO

-- 建立索引 --
CREATE NONCLUSTERED INDEX [ix_Phone]
    ON [dbo].[Stores]([Phone] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_CountryCode]
    ON [dbo].[Stores]([CountryCode] ASC);
GO

CREATE NONCLUSTERED INDEX [ix_ZipCode]
    ON [dbo].[Stores]([ZipCode] ASC);
GO